In [1]:
import sys
import io
import time
import requests
import datetime
import numpy as np
import pandas as pd

path_root = '../..'
sys.path.append(f'{path_root}/')
from src.util.functions import Functions as F

# Disable warning
pd.options.mode.chained_assignment = None 

level = 'regioni'

path_data = f'{path_root}/data'
path_csv = f'{path_data}/csv'

csv_out_name = f'{path_csv}/covid19-{level}.csv'
pop_regions_name = f'{path_csv}/popolazione-regioni.csv'
icu_regions_name = f'{path_csv}/terapie-intensive.csv'
regions_name = f'{path_csv}/regioni.csv'

In [2]:
url_cum = F.get_url_cum(level)
url_cum

'https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni.csv'

In [3]:
df_raw_r = pd.read_csv(url_cum)
df_raw_r

,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,...,note,ingressi_terapia_intensiva,note_test,note_casi,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2020-02-24T18:00:00,ITA,13,Abruzzo,42.351222,13.398438,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,ITA,17,Basilicata,40.639471,15.805148,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-02-24T18:00:00,ITA,18,Calabria,38.905976,16.594402,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-02-24T18:00:00,ITA,15,Campania,40.839566,14.250850,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-02-24T18:00:00,ITA,8,Emilia-Romagna,44.494367,11.341721,10,2,12,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8143,2021-03-17T17:00:00,ITA,19,Sicilia,38.115697,13.362357,734,116,850,14115,...,NaN,11.0,NaN,NaN,162710.0,0.0,1882197.0,965988.0,ITG,ITG1
8144,2021-03-17T17:00:00,ITA,9,Toscana,43.769231,11.255889,1453,241,1694,23213,...,NaN,21.0,NaN,NaN,175386.0,1224.0,2709044.0,395984.0,ITI,ITI1
8145,2021-03-17T17:00:00,ITA,10,Umbria,43.106758,12.388247,406,79,485,5419,...,"Si fa presente che 8 dei ricoveri NON UTI, non...",6.0,NaN,NaN,48526.0,0.0,748723.0,160434.0,ITI,ITI2
8146,2021-03-17T17:00:00,ITA,2,Valle d'Aosta,45.737503,7.320149,15,2,17,352,...,NaN,0.0,NaN,NaN,8414.0,0.0,77842.0,7453.0,ITC,ITC2


In [4]:
list(df_raw_r.columns)

['data',
 'stato',
 'codice_regione',
 'denominazione_regione',
 'lat',
 'long',
 'ricoverati_con_sintomi',
 'terapia_intensiva',
 'totale_ospedalizzati',
 'isolamento_domiciliare',
 'totale_positivi',
 'variazione_totale_positivi',
 'nuovi_positivi',
 'dimessi_guariti',
 'deceduti',
 'casi_da_sospetto_diagnostico',
 'casi_da_screening',
 'totale_casi',
 'tamponi',
 'casi_testati',
 'note',
 'ingressi_terapia_intensiva',
 'note_test',
 'note_casi',
 'totale_positivi_test_molecolare',
 'totale_positivi_test_antigenico_rapido',
 'tamponi_test_molecolare',
 'tamponi_test_antigenico_rapido',
 'codice_nuts_1',
 'codice_nuts_2']

In [5]:
# Drop useless columns
cols_to_drop = ['stato', 'lat', 'long', 'casi_da_sospetto_diagnostico', 'casi_da_screening', 'note', 'note_test', 'note_casi']
df_rd = df_raw_r.drop(cols_to_drop, axis=1)
df_rd

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2020-02-24T18:00:00,13,Abruzzo,0,0,0,0,0,0,0,...,0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,17,Basilicata,0,0,0,0,0,0,0,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-02-24T18:00:00,18,Calabria,0,0,0,0,0,0,0,...,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-02-24T18:00:00,15,Campania,0,0,0,0,0,0,0,...,0,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-02-24T18:00:00,8,Emilia-Romagna,10,2,12,6,18,0,18,...,18,148,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8143,2021-03-17T17:00:00,19,Sicilia,734,116,850,14115,14965,189,782,...,162710,2848185,1210951.0,11.0,162710.0,0.0,1882197.0,965988.0,ITG,ITG1
8144,2021-03-17T17:00:00,9,Toscana,1453,241,1694,23213,24907,12,1275,...,176610,3105028,1634285.0,21.0,175386.0,1224.0,2709044.0,395984.0,ITI,ITI1
8145,2021-03-17T17:00:00,10,Umbria,406,79,485,5419,5904,-1,324,...,48526,909157,327622.0,6.0,48526.0,0.0,748723.0,160434.0,ITI,ITI2
8146,2021-03-17T17:00:00,2,Valle d'Aosta,15,2,17,352,369,24,49,...,8414,85295,47398.0,0.0,8414.0,0.0,77842.0,7453.0,ITC,ITC2


In [6]:
# Compute most recent couple of dates
last_date_str = df_rd.iloc[-1].data.split('T')[0]
last_date = datetime.datetime.strptime(last_date_str, '%Y-%m-%d')
prev_date = F.get_previous_date(last_date)
prev_date_str = datetime.datetime.strftime(prev_date, '%Y-%m-%d')
last_date_str, prev_date_str

('2021-03-17', '2021-03-16')

In [7]:
# Current state S
df_raw_r_1 = df_rd[df_rd.data.str.startswith(last_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_1

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2021-03-17T17:00:00,1,Piemonte,3281,315,3596,27798,31394,610,2374,...,279669,2967126,1398404.0,28.0,269465.0,10204.0,2238463.0,728663.0,ITC,ITC1
1,2021-03-17T17:00:00,2,Valle d'Aosta,15,2,17,352,369,24,49,...,8414,85295,47398.0,0.0,8414.0,0.0,77842.0,7453.0,ITC,ITC2
2,2021-03-17T17:00:00,3,Lombardia,6641,781,7422,88343,95765,-11,4490,...,677948,7463075,3341526.0,81.0,655859.0,22089.0,6808283.0,654792.0,ITC,ITC4
3,2021-03-17T17:00:00,5,Veneto,1337,191,1528,34914,36442,729,2191,...,359247,5662414,1478907.0,24.0,351818.0,7429.0,4291988.0,1370426.0,ITH,ITH3
4,2021-03-17T17:00:00,6,Friuli Venezia Giulia,538,76,614,14388,15002,228,986,...,88203,1495255,535476.0,5.0,78958.0,9245.0,1350481.0,144774.0,ITH,ITH4
5,2021-03-17T17:00:00,7,Liguria,574,63,637,5530,6167,49,321,...,83550,1143852,482547.0,4.0,83550.0,0.0,996899.0,146953.0,ITC,ITC3
6,2021-03-17T17:00:00,8,Emilia-Romagna,3542,381,3923,65692,69615,623,2026,...,306324,4436998,1615543.0,31.0,306130.0,194.0,3745487.0,691511.0,ITH,ITH5
7,2021-03-17T17:00:00,9,Toscana,1453,241,1694,23213,24907,12,1275,...,176610,3105028,1634285.0,21.0,175386.0,1224.0,2709044.0,395984.0,ITI,ITI1
8,2021-03-17T17:00:00,10,Umbria,406,79,485,5419,5904,-1,324,...,48526,909157,327622.0,6.0,48526.0,0.0,748723.0,160434.0,ITI,ITI2
9,2021-03-17T17:00:00,11,Marche,747,133,880,9237,10117,-35,856,...,79975,934546,570398.0,11.0,79975.0,0.0,846532.0,88014.0,ITI,ITI3


In [8]:
# State S - 1
df_raw_r_0 = df_rd[df_rd.data.str.startswith(prev_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_0

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2021-03-16T17:00:00,1,Piemonte,3169,299,3468,27316,30784,584,2074,...,277295,2940674,1388528.0,25.0,267490.0,9805.0,2225948.0,714726.0,ITC,ITC1
1,2021-03-16T17:00:00,2,Valle d'Aosta,14,3,17,328,345,12,24,...,8365,84393,46940.0,0.0,8365.0,0.0,77496.0,6897.0,ITC,ITC2
2,2021-03-16T17:00:00,3,Lombardia,6474,765,7239,88537,95776,2613,4235,...,673458,7404066,3327916.0,83.0,652337.0,21121.0,6769491.0,634575.0,ITC,ITC4
3,2021-03-16T17:00:00,5,Veneto,1282,186,1468,34245,35713,389,1901,...,357056,5620569,1472876.0,32.0,349964.0,7092.0,4274911.0,1345658.0,ITH,ITH3
4,2021-03-16T17:00:00,6,Friuli Venezia Giulia,530,77,607,14167,14774,336,672,...,87217,1481351,530744.0,13.0,78320.0,8897.0,1340614.0,140737.0,ITH,ITH4
5,2021-03-16T17:00:00,7,Liguria,570,65,635,5483,6118,-219,343,...,83229,1136081,479721.0,5.0,83229.0,0.0,992106.0,143975.0,ITC,ITC3
6,2021-03-16T17:00:00,8,Emilia-Romagna,3516,378,3894,65098,68992,699,2184,...,304314,4399542,1609388.0,38.0,304122.0,192.0,3723069.0,676473.0,ITH,ITH5
7,2021-03-16T17:00:00,9,Toscana,1430,239,1669,23226,24895,329,1247,...,175335,3080090,1623906.0,15.0,174165.0,1170.0,2692646.0,387444.0,ITI,ITI1
8,2021-03-16T17:00:00,10,Umbria,402,78,480,5425,5905,-82,159,...,48202,902608,326669.0,7.0,48202.0,0.0,745359.0,157249.0,ITI,ITI2
9,2021-03-16T17:00:00,11,Marche,735,133,868,9284,10152,-290,529,...,79119,926935,565765.0,9.0,79119.0,0.0,840623.0,86312.0,ITI,ITI3


In [9]:
# New df regions state S with Trentino-Alto Adige
df_r_1 = df_raw_r_1
# df_r_1 = Functions.replace_bt_with_taa(df_raw_r_1)
# df_r_1.sort_values(by='codice_regione').reset_index(drop=True)

In [10]:
# New df regions state S - 1 with Trentino-Alto Adige
df_r_0 = df_raw_r_0
# df_r_0 = Functions.replace_bt_with_taa(df_raw_r_0)
# df_r_0.sort_values(by='codice_regione').reset_index()

In [11]:
df_r = df_r_1.copy()
# Merge with population data
df_pop_r = pd.read_csv(pop_regions_name)
df_m_r_1 = df_r.merge(df_pop_r, on='denominazione_regione')
df_m_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2,popolazione
0,2021-03-17T17:00:00,1,Piemonte,3281,315,3596,27798,31394,610,2374,...,2967126,1398404.0,28.0,269465.0,10204.0,2238463.0,728663.0,ITC,ITC1,4.341375e+06
1,2021-03-17T17:00:00,2,Valle d'Aosta,15,2,17,352,369,24,49,...,85295,47398.0,0.0,8414.0,0.0,77842.0,7453.0,ITC,ITC2,1.255010e+05
2,2021-03-17T17:00:00,3,Lombardia,6641,781,7422,88343,95765,-11,4490,...,7463075,3341526.0,81.0,655859.0,22089.0,6808283.0,654792.0,ITC,ITC4,1.010397e+07
3,2021-03-17T17:00:00,5,Veneto,1337,191,1528,34914,36442,729,2191,...,5662414,1478907.0,24.0,351818.0,7429.0,4291988.0,1370426.0,ITH,ITH3,4.907704e+06
4,2021-03-17T17:00:00,6,Friuli Venezia Giulia,538,76,614,14388,15002,228,986,...,1495255,535476.0,5.0,78958.0,9245.0,1350481.0,144774.0,ITH,ITH4,1.211357e+06
5,2021-03-17T17:00:00,7,Liguria,574,63,637,5530,6167,49,321,...,1143852,482547.0,4.0,83550.0,0.0,996899.0,146953.0,ITC,ITC3,1.543127e+06
6,2021-03-17T17:00:00,8,Emilia-Romagna,3542,381,3923,65692,69615,623,2026,...,4436998,1615543.0,31.0,306130.0,194.0,3745487.0,691511.0,ITH,ITH5,4.467118e+06
7,2021-03-17T17:00:00,9,Toscana,1453,241,1694,23213,24907,12,1275,...,3105028,1634285.0,21.0,175386.0,1224.0,2709044.0,395984.0,ITI,ITI1,3.722729e+06
8,2021-03-17T17:00:00,10,Umbria,406,79,485,5419,5904,-1,324,...,909157,327622.0,6.0,48526.0,0.0,748723.0,160434.0,ITI,ITI2,8.802850e+05
9,2021-03-17T17:00:00,11,Marche,747,133,880,9237,10117,-35,856,...,934546,570398.0,11.0,79975.0,0.0,846532.0,88014.0,ITI,ITI3,1.518400e+06


In [12]:
# Update icu csv data
df_icu_updated_r = pd.DataFrame.from_dict(F.icu_data())
df_icu_updated_r.to_csv(icu_regions_name, index=False)

In [13]:
# Merge with icu data
df_icu_r = pd.read_csv(icu_regions_name)
df_m_r_2 = df_m_r_1.merge(df_icu_r, left_on='denominazione_regione', right_on='regione')
df_m_r_2.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2,popolazione,regione,ricoverati_area_non_critica,posti_letto_area_non_critica,ricoverati_terapia_intensiva,posti_letto_terapia_intensiva,posti_letto_terapia_intensiva_attivabili
0,2021-03-17T17:00:00,1,Piemonte,3281,315,3596,27798,31394,610,2374,...,728663.0,ITC,ITC1,4.341375e+06,Piemonte,3169,5824,299,628,99
1,2021-03-17T17:00:00,2,Valle d'Aosta,15,2,17,352,369,24,49,...,7453.0,ITC,ITC2,1.255010e+05,Valle d'Aosta,14,227,3,20,12
2,2021-03-17T17:00:00,3,Lombardia,6641,781,7422,88343,95765,-11,4490,...,654792.0,ITC,ITC4,1.010397e+07,Lombardia,6474,12853,765,1416,114
3,2021-03-17T17:00:00,5,Veneto,1337,191,1528,34914,36442,729,2191,...,1370426.0,ITH,ITH3,4.907704e+06,Veneto,1282,6000,186,1000,0
4,2021-03-17T17:00:00,6,Friuli Venezia Giulia,538,76,614,14388,15002,228,986,...,144774.0,ITH,ITH4,1.211357e+06,Friuli Venezia Giulia,530,1277,77,175,0
5,2021-03-17T17:00:00,7,Liguria,574,63,637,5530,6167,49,321,...,146953.0,ITC,ITC3,1.543127e+06,Liguria,570,1796,65,222,12
6,2021-03-17T17:00:00,8,Emilia-Romagna,3542,381,3923,65692,69615,623,2026,...,691511.0,ITH,ITH5,4.467118e+06,Emilia-Romagna,3516,6490,378,760,0
7,2021-03-17T17:00:00,9,Toscana,1453,241,1694,23213,24907,12,1275,...,395984.0,ITI,ITI1,3.722729e+06,Toscana,1430,5033,239,598,18
8,2021-03-17T17:00:00,10,Umbria,406,79,485,5419,5904,-1,324,...,160434.0,ITI,ITI2,8.802850e+05,Umbria,402,860,78,139,2
9,2021-03-17T17:00:00,11,Marche,747,133,880,9237,10117,-35,856,...,88014.0,ITI,ITI3,1.518400e+06,Marche,735,1176,133,233,40


In [14]:
# Copy last merged df into df_r
df_r = df_m_r_2.copy()

In [15]:
# Add columns
df_r['variazione_ricoverati_con_sintomi'] = df_r_1['ricoverati_con_sintomi'] - df_r_0['ricoverati_con_sintomi']
df_r['variazione_terapia_intensiva'] = df_r_1['terapia_intensiva'] - df_r_0['terapia_intensiva']
df_r['variazione_totale_ospedalizzati'] = df_r_1['totale_ospedalizzati'] - df_r_0['totale_ospedalizzati']
df_r['posti_disponibili_terapia_intensiva'] = df_r['posti_letto_terapia_intensiva'] - df_r['terapia_intensiva']
df_r['saturazione_terapia_intensiva'] = ((df_r_1['terapia_intensiva'] / df_r['posti_letto_terapia_intensiva']) * 100).round(2)
df_r['variazione_isolamento_domiciliare'] = df_r_1['isolamento_domiciliare'] - df_r_0['isolamento_domiciliare']
df_r['variazione_nuovi_positivi'] = df_r_1['nuovi_positivi'] - df_r_0['nuovi_positivi']
df_r['nuovi_tamponi'] = df_r_1['tamponi'] - df_r_0['tamponi']
df_r['nuovi_casi_testati'] = df_r_1['casi_testati'] - df_r_0['casi_testati']
df_r['nuovi_deceduti'] = df_r_1['deceduti'] - df_r_0['deceduti']
df_r['nuovi_dimessi_guariti'] = df_r_1['dimessi_guariti'] - df_r_0['dimessi_guariti']
df_r['np_su_nt'] = ((df_r['nuovi_positivi'] / df_r['nuovi_tamponi']) * 100).round(2)
df_r['np_su_p'] = ((df_r['nuovi_positivi'] / df_r['popolazione']) * 100).round(5)
df_r['tc_su_p'] = ((df_r['totale_casi'] / df_r['popolazione']) * 100).round(2)

df_r = df_r.sort_values(by='nuovi_positivi', ascending=False).reset_index(drop=True)
df_r

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,saturazione_terapia_intensiva,variazione_isolamento_domiciliare,variazione_nuovi_positivi,nuovi_tamponi,nuovi_casi_testati,nuovi_deceduti,nuovi_dimessi_guariti,np_su_nt,np_su_p,tc_su_p
0,2021-03-17T17:00:00,3,Lombardia,6641,781,7422,88343,95765,-11,4490,...,55.16,-194,255,59009,13610.0,79,4422,7.61,0.04444,6.71
1,2021-03-17T17:00:00,15,Campania,1541,161,1702,96945,98647,574,2665,...,25.97,597,9,22392,13245.0,33,2058,11.90,0.04606,5.37
2,2021-03-17T17:00:00,1,Piemonte,3281,315,3596,27798,31394,610,2374,...,50.16,482,300,26452,9876.0,41,1723,8.97,0.05468,6.44
3,2021-03-17T17:00:00,5,Veneto,1337,191,1528,34914,36442,729,2191,...,19.10,669,290,41845,6031.0,59,1403,5.24,0.04464,7.32
4,2021-03-17T17:00:00,8,Emilia-Romagna,3542,381,3923,65692,69615,623,2026,...,50.13,594,-158,37456,6155.0,52,1335,5.41,0.04535,6.86
5,2021-03-17T17:00:00,16,Puglia,1592,218,1810,38008,39818,604,1734,...,38.31,544,608,12471,7209.0,34,1096,13.90,0.04326,4.22
6,2021-03-17T17:00:00,12,Lazio,2408,296,2704,41575,44279,-15,1728,...,31.39,-26,231,39100,22251.0,20,1723,4.42,0.02946,4.44
7,2021-03-17T17:00:00,9,Toscana,1453,241,1694,23213,24907,12,1275,...,40.30,-13,28,24938,10379.0,21,1242,5.11,0.03425,4.74
8,2021-03-17T17:00:00,6,Friuli Venezia Giulia,538,76,614,14388,15002,228,986,...,43.43,221,314,13904,4732.0,15,743,7.09,0.08140,7.28
9,2021-03-17T17:00:00,11,Marche,747,133,880,9237,10117,-35,856,...,57.08,-47,327,7611,4633.0,15,876,11.25,0.05638,5.27


In [16]:
# Write pre-processed data
df_r.to_csv(csv_out_name, index=False)

In [17]:
# ICU Stats
df_r[['denominazione_regione', 'terapia_intensiva', 'posti_disponibili_terapia_intensiva', 'posti_letto_terapia_intensiva', 'saturazione_terapia_intensiva']] \
    .sort_values(by='saturazione_terapia_intensiva', ascending=False).reset_index(drop=True)

,denominazione_regione,terapia_intensiva,posti_disponibili_terapia_intensiva,posti_letto_terapia_intensiva,saturazione_terapia_intensiva
0,P.A. Trento,54,36,90,60.00
1,Marche,133,100,233,57.08
2,Umbria,79,60,139,56.83
3,Lombardia,781,635,1416,55.16
4,Piemonte,315,313,628,50.16
5,Emilia-Romagna,381,379,760,50.13
6,Molise,17,22,39,43.59
7,Friuli Venezia Giulia,76,99,175,43.43
8,Abruzzo,89,126,215,41.40
9,Toscana,241,357,598,40.30


In [18]:
def add_cols(df):
    df['variazione_terapia_intensiva'] = df['terapia_intensiva'].diff()
    df['variazione_ricoverati_con_sintomi'] = df['ricoverati_con_sintomi'].diff()
    df['variazione_totale_ospedalizzati'] = df['totale_ospedalizzati'].diff()
    df['nuovi_dimessi_guariti'] = df['dimessi_guariti'].diff()
    df['nuovi_deceduti'] = df['deceduti'].diff()
    df['nuovi_tamponi'] = df['tamponi'].diff()
    df['nuovi_casi_testati'] = df['casi_testati'].diff()
    df['variazione_isolamento_domiciliare'] = df['isolamento_domiciliare'].diff()
    df = df.iloc[1:]
    df['ti_su_to'] = ((df['terapia_intensiva'] / df['totale_ospedalizzati']).replace(np.inf, 0) * 100).round(2)
    df['to_su_tp'] = ((df['totale_ospedalizzati'] / df['totale_positivi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nt'] = ((df['nuovi_positivi'] / df['nuovi_tamponi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nct'] = ((df['nuovi_positivi'] / df['nuovi_casi_testati']).replace(np.inf, 0) * 100).round(2)
    df['var_ti_perc'] = ((df['variazione_terapia_intensiva'] / (df['terapia_intensiva'] - df['variazione_terapia_intensiva'])).replace(np.inf, 0) * 100).round(2)
    df['var_pos_perc'] = ((df['variazione_totale_positivi'] / (df['totale_positivi'] - df['variazione_totale_positivi'])).replace(np.inf, 0) * 100).round(2)
    return df.fillna(0)

In [19]:
# df_rd.sort_values(by='codice_regione').reset_index(drop=True)
df_rn = pd.read_csv(regions_name)
for region_code in df_rn.codice_regione:
    if region_code != 4: # TAA
        df_cur_region = df_rd[df_rd.codice_regione == region_code]
        region_name = df_cur_region.denominazione_regione.iloc[0].lower()
        csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
        df_cur_region = add_cols(df_cur_region)
        df_cur_region.to_csv(csv_out_region_name, index=False)
        print('Aggiornato {file}'.format(file=csv_out_region_name))

Aggiornato ../../data/csv/regioni/covid19-piemonte.csv
Aggiornato ../../data/csv/regioni/covid19-valle d'aosta.csv
Aggiornato ../../data/csv/regioni/covid19-lombardia.csv
Aggiornato ../../data/csv/regioni/covid19-veneto.csv
Aggiornato ../../data/csv/regioni/covid19-friuli venezia giulia.csv
Aggiornato ../../data/csv/regioni/covid19-liguria.csv
Aggiornato ../../data/csv/regioni/covid19-emilia-romagna.csv
Aggiornato ../../data/csv/regioni/covid19-toscana.csv
Aggiornato ../../data/csv/regioni/covid19-umbria.csv
Aggiornato ../../data/csv/regioni/covid19-marche.csv
Aggiornato ../../data/csv/regioni/covid19-lazio.csv
Aggiornato ../../data/csv/regioni/covid19-abruzzo.csv
Aggiornato ../../data/csv/regioni/covid19-molise.csv
Aggiornato ../../data/csv/regioni/covid19-campania.csv
Aggiornato ../../data/csv/regioni/covid19-puglia.csv
Aggiornato ../../data/csv/regioni/covid19-basilicata.csv
Aggiornato ../../data/csv/regioni/covid19-calabria.csv
Aggiornato ../../data/csv/regioni/covid19-sicilia.csv


In [20]:
# df_tb = df_rd[df_rd.codice_regione > 20].reset_index(drop=True)
# df_tb

In [21]:
# Initialize df_taa
# df_taa = Functions.replace_bt_with_taa(df_tb[:2])
# for i in range(2, df_tb.shape[0], 2):
#     df_taa = df_taa.append(Functions.replace_bt_with_taa(df_tb[i:i+2]), ignore_index=True)
# df_taa = add_cols(df_taa)
# df_taa

In [22]:
# region_name = df_taa.denominazione_regione.iloc[0].lower()
# csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
# df_taa.to_csv(csv_out_region_name, index=False)
# print('Aggiornato {file}'.format(file=csv_out_region_name))